## Requirements

In [ ]:
!pip install fuzzywuzzy
!pip install imageio==2.4.1
!pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 4.9 MB/s 
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303885 sha256=6b22ea68a99d79dea30244bf3e9dc3d55fb48a2a7e4029ad841f932bf9fe113d
  Stored in directory: /root/.cache/pip/wheels/46/20/07/7bb9c8c44e6ec2efa60fd0e6280094f53f65f41767ef69a5ee
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 2.5 MB/s 


In [ ]:
!pip install PyQt5 nltk pyLDAvis
!pip install numba==0.48
!pip install librosa==0.6
import librosa
!pip install amfm_decompy aubio pydub
!pip install Wave moviepy==0.2.1.1 SpeechRecognition
!pip install sentence-transformers
!pip install wordninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.4 MB 5.1 MB/s 
     |████████████████████████████████| 1.7 MB 58.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 344 kB 48.8 MB/s 
     |████████████████████████████████| 59.9 MB 91 kB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=e9da1f5c2bac6e5027d201614b1b09207c7f506d1850b8a825dd4451f0c7151f
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=191039866516fb502d3fdaff99f5a47114805f5f82a387e5656c0aea713e10da
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f

ModuleNotFoundError: ignored

In [ ]:
!pip install gensim docx2txt
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords# Import the library
# to convert MSword doc to txt for processing.
import docx2txt
!pip install textract PyPDF2
import textract
import PyPDF2
!pip install jsonlines
!pip install pyyaml==5.4.1

In [ ]:
#spacy
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy
import pyLDAvis.gensim_models
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import jsonlines

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

#warning
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
!wget https://raw.githubusercontent.com/pandmi/jobzilla_ai/main/jz_skill_patterns.jsonl

In [ ]:
import en_core_web_sm
nlp = spacy.load("en_core_web_sm")
skill_pattern_path = "/content/jz_skill_patterns.jsonl"
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

## Input

In [ ]:
import os
os.environ['GOOGLE_CLOUD_PROJECT']="august-cirrus-306813"

In [ ]:
from google.colab import auth
import gspread
from google.auth import default
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
#defining my worksheet
worksheet = gc.open('Outsourcing Tracker').sheet1
#get_all_values gives a list of rows
rows = worksheet.get_all_values()
#Convert to a DataFrame 
df = pd.DataFrame(rows)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.head()

In [ ]:
for i in range(7):
  dir= "question_answers"+str(i+1)
  %mkdir $dir
  %cd $dir
  file_id=df.iloc[i]['Response'].split("/")[-2]
  !gdown $file_id
  %cd ../ 

## Calculations:

In [ ]:
#ideal answers
ideal={}
for i in range(len(df['Ideal_answer'])-1):
  ideal[i+1]=df.iloc[i]['Ideal_answer']

In [ ]:
ideal

In [ ]:
#content_scoring flag
flag=df['Content_Scoring']
dat_format=df['Answer_Format']
base_flag=df['Base_Scoring ']
id={}
labels=['Interaction_ID','Candidate_ID','Test_ID']
for i in labels:
  id[i]=df.iloc[0][i]

In [ ]:
flag

In [ ]:
def extract_text_from_pdf(file):
    '''Opens and reads in a PDF file from path'''
    
    pdfFileObj = open(file, 'rb')
      
    # creating a pdf reader object
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
      
    # printing number of pages in pdf file
    page_count = pdfReader.getNumPages()
      
    # creating a page object
    pageObj = pdfReader.getPage(0)
      
    # extracting text from page
    text = [pdfReader.getPage(i).extractText() for i in range(page_count)]
      
    # closing the pdf file object
    pdfFileObj.close()
    
    
    
    return str(text).replace("\\n", "")

In [ ]:
import en_core_web_sm

# Load pre-trained English language model
#nlp = en_core_web_sm.load()

def create_tokenized_texts_list():
    '''Create two lists, one with the names of the candidate and one with the tokenized 
       resume texts extracted from either a .pdf or .doc'''
    resume_texts= ''
    
    # Loop over the contents of the directory containing the resumes, filtering by .pdf or .doc(x)
    resume=os.listdir('/content/question_answers7/')[0]

    # Read in every resume with pdf extension in the directory
    resume_texts=(extract_text_from_pdf('/content/question_answers7/' + resume))
    
        
    return resume_texts, resume


In [ ]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))

In [ ]:
import nltk
import wordninja
nltk.download('omw-1.4')
def clean_resume():
  texts,names=create_tokenized_texts_list()
  texts=' '.join(texts.split("\\uf"))
  clean = []

  review = re.sub(
      '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
      " ",
      texts,
  )
  review = review.lower()
  review = review.split()
  lm = WordNetLemmatizer()
  review = [
      lm.lemmatize(word)
      for word in review
      if not word in set(stopwords.words("english"))
  ]
  review = " ".join(review)
  clean=' '.join(wordninja.split(review))
  return clean

In [ ]:
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# get the match percentage
def resume_score():
  cleaned_resume=clean_resume()
  job_skills=df.iloc[0]['Description']
  text_list = [cleaned_resume,job_skills]
  cv = CountVectorizer()
  count_matrix = cv.fit_transform(text_list)
  matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
  matchPercentage = round(matchPercentage, 2)
  mp2=round(((fuzz.token_sort_ratio(cleaned_resume.lower(),job_skills.lower()))/len(job_skills.split(" ")))*100)


  return mp2 if mp2>matchPercentage else matchPercentage


In [ ]:
resume_sc=resume_score()
resume_sc

In [ ]:
def startConvertion(path = 'sample.wav',lang = 'en-IN'):
    with sr.AudioFile(path) as source:
        r = sr.Recognizer()
        print('Fetching File')
        audio_text = r.listen(source,timeout=6000, phrase_time_limit=6000)
        # recoginize_() method will throw a request error if the API is unreachable, hence using exception handling
        try:
        
            # using google speech recognition
            print('Converting audio transcripts into text ...')
            text = r.recognize_google(audio_text)
            print(text)
            return text
    
        except:
            print('Sorry.. run again...')

Transcript Grammer Score

In [ ]:
!pip install -U git+https://github.com/PrithivirajDamodaran/Gramformer.git
from gramformer import Gramformer
import torch
!python -m spacy download en
import spacy
spacy.load("en")

In [ ]:
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
set_seed(1212)
gf = Gramformer(models = 1, use_gpu=False)

In [ ]:
def Grammer_score(net_text):
  percent_error = []
  s = 0
  for i in net_text:
    influent_sentence = i
    corrected_sentences = gf.correct(influent_sentence, max_candidates=1)
    print("[Input] ", influent_sentence)
    for corrected_sentence in corrected_sentences:
      print("corrected sentence: "+corrected_sentence)
      
    percent_error.append((len(gf.get_edits(influent_sentence, corrected_sentence))/len(influent_sentence.split()))*100)
    s+= percent_error
  return percent_error , s/(len(net_text))

In [ ]:
Grammer_score("hi, how is you")

In [ ]:
import numpy as np # linear algebra
import pandas as pd
import wave, math, contextlib
import speech_recognition as sr
#import imageio-ffmpeg
from moviepy.editor import AudioFileClip

#returns pace,duration,text,filenames
def audio_read(path):
  net_text=[]
  net_word_count=[]
  net_total_duration=[]
  net_transcribed_audio_file_name=[]
  for i in range(len(df['Response'])-1):
    transcribed_audio_file_name = "transcribed_speech"+str(i+1)+".wav"
    zoom_video_file_name = path[i]
    audioclip = AudioFileClip(zoom_video_file_name)
    audioclip.write_audiofile(transcribed_audio_file_name)
    with contextlib.closing(wave.open(transcribed_audio_file_name,'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
    total_duration = math.ceil(duration / 60)
    r = sr.Recognizer()
    with open('transcription.txt', 'w') as f:
      print(f.write(startConvertion(transcribed_audio_file_name)))
    with open('transcription.txt', 'r') as f:
      text=f.read()
    word_count_tex=len(text.split(" "))/(total_duration/2);
    #print("Words per minute are:",word_count_tex)
    net_text.append(text)
    net_word_count.append(word_count_tex)
    net_total_duration.append(total_duration)
    net_transcribed_audio_file_name.append(transcribed_audio_file_name)
  return net_text,net_word_count,net_total_duration,net_transcribed_audio_file_name

In [ ]:
#storing all the data
path=[]
for i in range(len(df['Response'])-1):
  dir='/content/question_answers'+str(i+1)+'/'
  newfile=os.listdir(dir)[0]
  new_dir=dir+newfile
  path.append(new_dir)

In [ ]:
path

In [ ]:
net_text,net_word_count,net_total_duration,net_transcribed_audio_file_name=audio_read(path)

In [ ]:
net_text[0],net_transcribed_audio_file_name[0],net_total_duration[0]

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
#print(stopwords.words('english'))
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
def filtered_words(net_text):
  net_filtered=[]
  net_filtered_unique=[]
  for i in net_text:
    word_tokens = word_tokenize(i)
    filtered_sentence = []
    for w in word_tokens:
      if w not in stop_words:
        filtered_sentence.append(w)
    net_filtered.append(filtered_sentence)
    filtered_sentence_unique=list(pd.Series(filtered_sentence).unique())
    net_filtered_unique.append(filtered_sentence_unique)
    #print(word_tokens)
    #print(filtered_sentence)
  return net_filtered,net_filtered_unique

In [ ]:
net_filtered,net_filtered_unique=filtered_words(net_text)

In [ ]:
from operator import itemgetter
import math
def total_length(net_text):
  net_total_words=[]
  for text in net_text:
    total_words = text.split()
    total_word_length = len(total_words)
    net_total_words.append(total_word_length)
  #print(total_word_length)
  return net_total_words

In [ ]:
net_total_length=total_length(net_text)
net_total_length

In [ ]:
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

In [ ]:
def tf_scoring(text,net_filtered_unique):
  net_total_length=total_length(net_text)
  tf_score = {}
  for each_word in ' '.join([' '.join(i) for i in net_filtered_unique]).split(" "):
      each_word = each_word.replace('.','')
      if each_word not in stop_words:
          if each_word in tf_score:
              tf_score[each_word] += 1
          else:
              tf_score[each_word] = 1

  # Dividing by total_word_length for each dictionary element
  tf_score.update((x, y/int(np.mean(net_total_length))) for x, y in tf_score.items())
  #print(tf_score)
  return tf_score

In [ ]:
def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result

In [ ]:
def get_power_words(net_text,net_filtered_unique):
  keymax=[]
  tf_val=get_top_n(tf_scoring(' '.join(net_text),net_filtered_unique), 20)
  keymax.extend(sorted(tf_val, key=tf_val.get, reverse=True))
  return list(set(list(keymax)))[:20]

In [ ]:
power_words=get_power_words(net_text,net_filtered_unique)

In [ ]:
import matplotlib
import pandas
import wordcloud
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd


def plot_cloud(net_text):
  text=' '.join(net_text)
  comment_words = ''
  stopwords = set(STOPWORDS)
  net_filtered,net_filtered_unique=filtered_words(net_text)
  # split the value
  tokens = ' '.join(([' '.join(i) for i in net_filtered_unique])).split(" ")

  # Converts each token into lowercase
  for i in range(len(tokens)):
      tokens[i] = tokens[i].lower()

      comment_words += " ".join(tokens)+" "

      wordcloud = WordCloud(width = 800, height = 800,background_color ='white',stopwords = stopwords,min_font_size= 10).generate(comment_words)
  plt.figure(figsize = (8, 8), facecolor = None)
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout(pad = 0)
  plt.show()
  
  return wordcloud

In [ ]:
 plot_cloud(net_text).to_file("word_cloud.png")

In [ ]:
def power_word_density(net_text):
  cou=[];
  total_duration=net_total_duration
  net_filtered,net_filtered_unique=filtered_words([val for key,val in ideal.items()])
  for i in net_text:
    for j in net_filtered_unique:
        cou.append(len(j)/len(i.split(" ")))
  
  cou=np.mean(cou)
  #print('Power word density/min:',cou)
  return cou

In [ ]:
power_density=power_word_density(net_text)
power_density

In [ ]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from scipy.signal import find_peaks
import librosa.display
import sklearn
def pitch_calc(path):
  ind=net_total_length.index(min(net_total_length))
  path_to_wav=path[ind]
  data, sampling_frequency = librosa.load(path_to_wav)
  auto = sm.tsa.acf(data, nlags=2000)
  peaks = find_peaks(auto)[0] # Find peaks of the autocorrelation
  lag = peaks[0] # Choose the first peak as our pitch component lag
  pitc=sampling_frequency / lag
  return pitc,peaks

In [ ]:
pitc,peaks=pitch_calc(path)

In [ ]:
pitc

In [ ]:
import math
def volume_calc(path):
  ind=net_total_length.index(min(net_total_length))
  path_to_wav=path[ind]
  y, sr = librosa.load(path_to_wav)
  second = []
  for s in range(0,len(y),sr):
      second.append( np.abs(y[s:s+sr]).mean())
  volume=[20 * math.log10(i) for i in second]
  return np.max(np.abs(volume))

In [ ]:
volume=volume_calc(path)

In [ ]:
!pip install -q transformers
from transformers import pipeline

def sentiment_happy_proba(net_text):
  happy_proba=[]
  classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
  for text in net_text:
    classify=classifier(text)
    df=pd.DataFrame(classify[0],columns=['label','score'])
    happy_proba.append(float(df[df.label=='joy']['score']))
  return happy_proba

In [ ]:
happy_proba=sentiment_happy_proba(net_text)
happy_proba

In [ ]:
def summary(net_text,flag):
  content_score=[]
  flag=list(flag)
  for i in range(len(flag)-1):
    matchPercentage=0
    mp2=0
    if flag[i]=='Yes':
      answer=ideal[(i+1)]
      text=net_text[i]
      text_list = [text,answer]
      cv = CountVectorizer()
      count_matrix = cv.fit_transform(text_list)
      matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
      matchPercentage = round(matchPercentage, 2)
      mp2=round(((fuzz.token_sort_ratio(text.lower(),answer.lower()))/len(answer.split(" ")))*100)
      content_score.append(mp2 if mp2>matchPercentage else matchPercentage)
    else:
      content_score.append('Not Applicable')
  return content_score

In [ ]:
content_score=summary(net_text,flag)

In [ ]:
from pydub import AudioSegment,silence

def silence_detect(path):
  silence_secs=[]
  silence_num=[]
  for i in path:
    myaudio = intro = AudioSegment.from_wav(i)
    dBFS=myaudio.dBFS
    silenced = silence.detect_silence(myaudio, min_silence_len=1000, silence_thresh=dBFS-16)

    silenced = [((stop/1000)-(start/1000)) for start,stop in silenced] #in sec
    #print(np.max(silenced),len(silenced))
    if silenced!=[]:
      silence_secs.append(np.max(silenced)),silence_num.append(len(silenced))
    else:
      silence_secs.append(0),silence_num.append(0)
  return silence_secs,silence_num

In [ ]:
silence_secs,silence_num=silence_detect(net_transcribed_audio_file_name)

In [ ]:
def filler_words_cohort(net_text):
  cohort=[]
  net_filtered,net_filtered_unique=filtered_words(net_text)
  for i in range(len(net_filtered)):
    cohort_score=len(net_filtered[i])/len(net_text[i].split(" "))
    if cohort_score>0.7:
      cohort.append(4)
    elif cohort_score>0.4:
      cohort.append(3)
    elif cohort_score>0.1:
      cohort.append(2)
    else:
      cohort.append(1)
  return cohort

In [ ]:
filler_words=filler_words_cohort(net_text)
filler_words

In [ ]:
def sentiment_cohort(net_text):
  happy_proba=sentiment_happy_proba(net_text)
  happy=[]
  for i in happy_proba: 
    if i>0.7:
      happy.append(4)
    elif i>0.4:
      happy.append(3)
    elif i>0.1:
      happy.append(2)
    else:
      happy.append(1)
  return happy

In [ ]:
sentiment=sentiment_cohort(net_text)

In [ ]:
def confidence_cohort(path):
  net=[]
  net_text,net_word_count,net_total_duration,net_transcribed_audio_file_name=audio_read(path)
  filler_words=filler_words_cohort(net_text)
  sentiment=sentiment_cohort(net_text)
  for i in range(len(sentiment)):
    cohort=(sentiment[i]+filler_words[i])/2
    cohort=int(cohort)
    if cohort==1:
      net.append('D')
    elif cohort==2:
      net.append('C')
    elif cohort==3:
      net.append('B')
    elif cohort==4:
      net.append('A')
  return net

In [ ]:
confidence=confidence_cohort(path)

In [ ]:
def pitch_volume_cohort(path):
  pitc,peaks=pitch_calc(path)
  volume=volume_calc(path)
  pitch=np.abs(pitc-670)
  p_cohort=0
  v_cohort=0
  if pitch<=500:
    p_cohort=4
  elif pitch<=1000:
    p_cohort=3
  elif pitch<=1500:
    p_cohort=2
  else:
    p_cohort=1
  if volume>=75:
    v_cohort=4
  elif volume>=50:
    v_cohort=3
  elif volume>=35:
    v_cohort=2
  else:
    v_cohort=1
  return math.ceil((p_cohort+v_cohort)/2)

In [ ]:
pitch_volum=pitch_volume_cohort(path)
pitch_volum

In [ ]:
def energy_cohort(path):
  cohort= pitch_volume_cohort(path)
  cohort=int(cohort)
  cohort=[cohort]*6
  net=[]
  for i in range(len(cohort)):
    if cohort[i]==1:
      net.append('D')
    elif cohort[i]==2:
      net.append('C')
    elif cohort[i]==3:
      net.append('B')
    elif cohort[i]==4:
      net.append('A')
  return net

In [ ]:
energy=energy_cohort(path)

In [ ]:
def fluency_cohort(path):
  net_text,net_word_count,net_total_duration,net_transcribed_audio_file_name=audio_read(path)
  net_filtered,net_filtered_unique=filtered_words(net_text)
  tex=''
  pace_cohort=[]
  for i in range(len(net_filtered)):
    if net_word_count[i]>170 and net_word_count[i]<190:
      pace_cohort.append(4)
    elif net_word_count[i]>100 and net_word_count[i]<120:
      pace_cohort.append(3)
    elif net_word_count[i]>50 and net_word_count[i]>90:
      pace_cohort.append(2)
    else:
      pace_cohort.append(1)
  silenc,coun=silence_detect(net_transcribed_audio_file_name)
  silence_cohort=[]
  for i in range(len(silenc)):
    if silenc[i] <3 or coun[i]<4:
      silence_cohort.append(4)
    elif silenc[i] <5 or coun[i]<8:
      silence_cohort.append(3)
    elif silenc[i] < 8 or coun[i]<10:
      silence_cohort.append(2)
    else:
      silence_cohort.append(1)
  net=[]
  for i in range(len(pace_cohort)):
    coh=(pace_cohort[i]+silence_cohort[i])//2
    if coh==4:
      net.append('A')
    elif coh==3:
      net.append('B')
    elif coh==2:
      net.append('C')
    elif coh==1:
      net.append('D')
  return net 

In [ ]:
fluency=fluency_cohort(path)

In [ ]:
fluency

In [ ]:
def aggregate_score(content_score,confidence,energy,fluency):
  content_score=[i for i in content_score if i!='Not Applicable']
  content_score=np.mean(content_score)
  cohort=0
  net=[]
  if content_score>20 and content_score<30:
    cohort=1
  elif content_score>30 and content_score<40:
    cohort=2
  elif content_score>40 and content_score<60:
    cohort=3
  elif content_score>60 and content_score<70:
    cohort=4
  for i in range(len(confidence)):
    net.append(((69-ord(confidence[i]))+(69-ord(energy[i]))+(69-ord(fluency[i]))+cohort)//4)
  return [chr(69-i)for i in net]

In [ ]:
content_score

In [ ]:
[ord(i) for i in confidence]

In [ ]:
fluency

In [ ]:
energy

In [ ]:
per_ques=aggregate_score(content_score,confidence,energy,fluency)
per_ques

In [ ]:
interaction_score=69-np.mean([ord(i) for i in per_ques])
interaction_score

In [ ]:
interaction_cohort=chr(69-round(interaction_score))
interaction_cohort

In [ ]:
quotients=pd.read_csv("/content/The Power words dashboard  - Power Words.csv").dropna()
text=' '.join(net_text)

In [ ]:
sales=' '.join(list(quotients['Sales Power words ']))
manager=' '.join(list(quotients['Manager Power Words ']))
leadership=' '.join(list(quotients['Leadership Power Words']))
hr=' '.join(list(quotients['HR Power Words ']))
learning=' '.join(list(quotients['Learning Quotient ']))
service=' '.join(list(quotients['Service Quotient ']))

In [ ]:
def quotient_scores(sales,manager,leadership,hr,learning,service,text):
  content_score={}
  sales_list = [text,sales]
  mg_list=[text,manager]
  ld_list=[text,leadership]
  hr_list=[text,hr]
  lr_list=[text,learning]
  sv_list=[text,service]
  cv = CountVectorizer()
  count_matrix = cv.fit_transform(sales_list)
  matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
  matchPercentage = round(matchPercentage, 2)
  mp2=round(((fuzz.token_sort_ratio(text.lower(),sales.lower()))/len(sales.split(" ")))*100)
  content_score['sales']=(mp2 if mp2>matchPercentage else matchPercentage)

  count_matrix = cv.fit_transform(mg_list)
  matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
  matchPercentage = round(matchPercentage, 2)
  mp2=round(((fuzz.token_sort_ratio(text.lower(),manager.lower()))/len(manager.split(" ")))*100)
  content_score['manager']=(mp2 if mp2>matchPercentage else matchPercentage)

  count_matrix = cv.fit_transform(ld_list)
  matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
  matchPercentage = round(matchPercentage, 2)
  mp2=round(((fuzz.token_sort_ratio(text.lower(),leadership.lower()))/len(leadership.split(" ")))*100)
  content_score['leadership']=(mp2 if mp2>matchPercentage else matchPercentage)

  count_matrix = cv.fit_transform(hr_list)
  matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
  matchPercentage = round(matchPercentage, 2)
  mp2=round(((fuzz.token_sort_ratio(text.lower(),hr.lower()))/len(hr.split(" ")))*100)
  content_score['hr']=(mp2 if mp2>matchPercentage else matchPercentage)
  
  count_matrix = cv.fit_transform(lr_list)
  matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
  matchPercentage = round(matchPercentage, 2)
  mp2=round(((fuzz.token_sort_ratio(text.lower(),learning.lower()))/len(learning.split(" ")))*100)
  content_score['learning']=(mp2 if mp2>matchPercentage else matchPercentage)

  count_matrix = cv.fit_transform(sv_list)
  matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
  matchPercentage = round(matchPercentage, 2)
  mp2=round(((fuzz.token_sort_ratio(text.lower(),service.lower()))/len(service.split(" ")))*100)
  content_score['service']=(mp2 if mp2>matchPercentage else matchPercentage)

  return content_score


In [ ]:
quotient_dict=quotient_scores(sales,manager,leadership,hr,learning,service,text)

In [ ]:
quotient_dict

In [ ]:
id

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
cloud=cv2.imread('/content/word_cloud.png')

In [ ]:
content_score=[i for i in content_score if i!='Not Applicable']
content_score=np.mean(content_score)

Grammer Score INIT

In [ ]:
percentage_grammer_score , avg_grammer_score = Grammer_score(net_text)

In [ ]:
#fill up ids
output={}
output['Candidate_ID']=[id['Candidate_ID']]*6
output['Interaction_ID']=[id['Interaction_ID']]*6
output['Test_ID']=[id['Test_ID']]*6
output['Interaction_Mode']=[df.iloc[0]['Interaction_Mode']]*6
output['MCQ_Percentage']=[(0 if output['Interaction_Mode']!='MCQ' else 1)]*6
output['Sales_Quotient']=[quotient_dict['sales']]*6
output['Manager_Quotient']=[quotient_dict['manager']]*6
output['Leadership_Quotient']=[quotient_dict['leadership']]*6
output['Learner_Quotient']=[quotient_dict['learning']]*6
output['People_Quotient']=[quotient_dict['service']]*6
output['Resume_Score']=[resume_sc]*6
output['Pace']=[np.mean(net_word_count)]*6
output['Power_word_density']=[power_density]*6
output['Word_Cloud']=[cloud]*6
output['Volume']=[volume]*6
output['Pitch']=[pitc]*6
output['Aggregate_content_score']=[content_score]*6
output['Raw_interaction_score']=[interaction_score]*6
output['Interaction_score']=[interaction_cohort]*6
output['Question_no.']=list(df['Question_ID'])[:-1]
output['MCQ_Value']=[0]*6
#pd.DataFrame(output,columns=output.keys(),index=[0])

#avg Grammer score
output['MCQ_Value']=[avg_grammer_score]*6

In [ ]:
Grammer output['Question_no.']

In [ ]:
content_score

In [ ]:
output['Transcript']=net_text
#percentage grammer score
output['Transcript_Grammer_Score']= percentage_grammer_score 
output['Confidence']=confidence
output['Fluency']=fluency
output['Energy']=energy
output['Content_score']=content_score
output['per_question_score']=per_ques
output['silence_number'],output['silence_length']=silence_secs,silence_num
output['filler_words_score']=filler_words_cohort(net_text)
output['sentiment_score']=happy_proba

In [ ]:
import collections.abc

for key,val in output.items():
  if isinstance(val, collections.abc.Sequence):
    print(key,len(val))
  else:
    print('no len for ',key)

In [ ]:
output=pd.DataFrame(output,columns=output.keys())

In [ ]:
cv2_imshow(output['Word_Cloud'][0])

In [ ]:
output